In [0]:
import librosa
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import csv

# Preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Keras
import tensorflow as tf
import keras

In [0]:
!rm -rf __MACOSX
# !rm friends_clean.zip
!rm -rf friends_clean
!rm data.csv

In [178]:
# Creating dataset
header = 'filename chroma_stft rmse spectral_centroid spectral_bandwidth rolloff zero_crossing_rate'
for i in range(1, 21):
  header +=  f' mfcc{i}'
header += ' label'
header = header.split()

# Calculating the features

!unzip merged_clean.zip

file = open('data.csv', 'w')
with file:
  writer = csv.writer(file)
  writer.writerow(header)
  
songs = ["Havana","Closer","Shape of You","Wake Me Up","Despacito","Eastside","Sunflower","Friends","Lights Down Low","New Rules"]
# songs = 'havana closer shape wake despacito eastside sunflower friends lights newrules'.split()
# songs=["Havana","Closer","Shape of You","Despacito","Sunflower","Friends","New Rules"]

for s in songs:
  print (s)
  for filename in os.listdir(f'./merged_clean/{s}'):
    if filename[0] != '.':
      print(filename)
      songname = f'./merged_clean/{s}/{filename}'
#       print(songname)
      y, sr = librosa.load(songname, mono=True, duration=5)
      chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr)
      rmse = librosa.feature.rmse(y=y)
      spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr)
      spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr)
      rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr)
      zcr = librosa.feature.zero_crossing_rate(y)
      mfcc = librosa.feature.mfcc(y=y, sr=sr)
      to_append = [filename,np.mean(chroma_stft),np.mean(rmse),np.mean(spec_cent),np.mean(spec_bw),np.mean(rolloff),np.mean(zcr)]
      for e in mfcc:
        to_append.append(np.mean(e))
      to_append.append(s)
      file = open('data.csv', 'a')
      with file:
        writer = csv.writer(file)
        writer.writerow(to_append)

Archive:  merged_clean.zip
   creating: merged_clean/
   creating: merged_clean/Wake Me Up/
  inflating: merged_clean/Wake Me Up/Wake Me Up_100.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_112.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_111.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_198.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_199.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_25.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_197.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_26.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_243.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_109.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_244.mp3  
  inflating: merged_clean/Wake Me Up/Wake Me Up_245.mp3  
   creating: merged_clean/New Rules/
  inflating: merged_clean/New Rules/New Rules_69.mp3  
  inflating: merged_clean/New Rules/New Rules_169.mp3  
  inflating: merged_clean/New Rules/New Rules_68.mp3  
  inflating

In [179]:
# Reading dataset
data = pd.read_csv('data.csv')
data.head()

# Dropping uneccessary columns
data = data.drop(['filename'], axis=1)
data.head()

# Encode genres into integers
song_list = data.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(song_list)
print(y)

# normalizing the dataset
scaler = StandardScaler()
X = scaler.fit_transform(np.array(data.iloc[:, :-1], dtype=float))

print(X)

# splitting the dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

[4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 7 7 7
 7 7 7 7 7 7 7 7 7 7 7 7 7 9 9 9 9 9 9 9 9 9 9 9 9 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 2 2 2 2 2 2 2 2 8 8 8 8 8 8 8 8 8 8 8 8 8 8 3 3 3 3 3 3 3 3 3 3 3 5 5
 5 5 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6]
[[ 1.45810441 -0.89060442  1.86531221 ...  0.15311002 -0.49308158
   0.00541031]
 [ 0.26752766 -0.51504481  2.20830024 ... -1.14892355 -0.50322675
  -1.16471675]
 [ 1.54872714 -1.14412853  0.24339055 ...  0.08340543  0.19888734
  -0.47352709]
 ...
 [-0.83047472  0.01153176  1.06309961 ... -0.64289791 -0.19218618
  -1.20439302]
 [ 0.57179615 -1.56474349 -0.06957231 ...  0.05972365 -0.11866977
  -1.6440578 ]
 [ 0.17046191  0.75624951 -1.68453779 ...  1.26211722  0.13829702
  -0.58825256]]


In [186]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics, model_selection


# instantiate the model (using the default parameters)
clf = KNeighborsClassifier(n_neighbors=10)
# print(X_train,y_train)

# clf.fit(X_train,y_train)
print(X.shape,y.shape)
# print(X_train.shape,y_train.shape,X_test.shape)
# y_pred=clf.predict(X_test)
# print(y_test)
# print(y_pred)
accs=model_selection.cross_val_score(clf,X, y,cv=11)
# print(accs)
print("Avg Accuracy (cv=5):", sum(accs)/len(accs))


# print(clf.predict_proba(X_test))



(135, 26) (135,)
Avg Accuracy (cv=5): 0.11720618953239276


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=11.
  % (min_groups, self.n_splits)), Warning)


In [28]:
from keras import models
from keras import layers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_shape=(X_train.shape[1],)))

model.add(layers.Dense(128, activation='relu'))

model.add(layers.Dense(64, activation='relu'))

model.add(layers.Dense(10, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
              
history = model.fit(X_train,
                    y_train,
                    epochs=20,
                    batch_size=128)
                    
# calculate accuracy
test_loss, test_acc = model.evaluate(X_test,y_test)
print('test_acc: ',test_acc)

# predictions
predictions = model.predict(X_test)
np.argmax(predictions[0])

Epoch 1/20
48/48 [==============================] - 0s 5ms/step - loss: 2.3079 - acc: 0.1250
Epoch 2/20
48/48 [==============================] - 0s 136us/step - loss: 2.2147 - acc: 0.2500
Epoch 3/20
48/48 [==============================] - 0s 126us/step - loss: 2.1369 - acc: 0.4583
Epoch 4/20
48/48 [==============================] - 0s 112us/step - loss: 2.0665 - acc: 0.4167
Epoch 5/20
48/48 [==============================] - 0s 108us/step - loss: 2.0033 - acc: 0.5208
Epoch 6/20
48/48 [==============================] - 0s 115us/step - loss: 1.9420 - acc: 0.5417
Epoch 7/20
48/48 [==============================] - 0s 95us/step - loss: 1.8821 - acc: 0.6042
Epoch 8/20
48/48 [==============================] - 0s 94us/step - loss: 1.8218 - acc: 0.6250
Epoch 9/20
48/48 [==============================] - 0s 99us/step - loss: 1.7617 - acc: 0.6875
Epoch 10/20
48/48 [==============================] - 0s 110us/step - loss: 1.7008 - acc: 0.7083
Epoch 11/20
48/48 [==============================] - 0

1

In [0]:
from sklearn.cluster import KMeans
from sklearn import metrics

# instantiate the model (using the default parameters)
clf = KMeans(n_clusters=10)

clf.fit(X)
print(clf.labels_)
print(clf.cluster_centers_)
print(y)

# clf.fit(X_train)
# preds=clf.predict(X_test)

print("Accuracy:", metrics.accuracy_score(clf.labels_, y))

[3 0 1 2 9 0 6 4 1 8 0 1 0 7 1 3 0 2 2 4 8 5 4 1 2 4 7 6 0 1]
[[-0.91390885  0.08459118 -0.83527492 -0.6987232  -0.95221587 -0.45043773
   0.29853764  0.45326186 -0.50956928 -0.28841137  0.98216531 -0.12370633
  -0.31359027 -0.52219342 -0.15046082  1.20853162 -0.8106709  -0.12417226
   0.32505768  0.35080504 -0.11779997 -0.85311361 -0.29345728  0.56050884
  -0.38495076 -0.96538426]
 [ 0.83879255  0.38972987 -0.15510446  0.26713517 -0.02975791 -0.2248835
   0.89930245  0.78482737  0.12604726 -0.84396386  0.16053692  1.57982727
   1.13493203  1.63973141 -1.06187068 -0.87594338  1.41623393  0.98057553
  -1.57865383 -0.39010384  0.24469522  0.36710478  0.24701487  0.0352698
  -0.73941558 -0.10019199]
 [ 0.43327845 -1.35985445 -0.39153648 -1.10136781 -0.3599169   0.6915692
  -1.56658217  0.93423895 -0.80688588  1.53269681 -0.2460558  -0.87976193
  -0.92491761 -0.45562547 -0.4990115   0.25682391  0.69926547 -0.52282503
   0.31906592 -0.6243438   0.59263525 -0.47478136 -1.41193053  0.41442559